# Megatron 直接使用预训练模型进行预测 (BPE v2)

## 环境准备

准备运行这个笔记本的 Jupyter kernel(**如果已经准备就绪，不要重复执行！**)：


1. 配置一个 Conda 环境作为 Jupyter Kernel

In [ ]:
%conda env update -f environments/environment-ipy.yml

安装完毕后，为该 Notebook 选择这个 Kernel (名为`Megatron_LM-ipy`)

2. 在Kernel所在 Conda 环境中安装 Apex

需要通过 pip 从 github 下载源代码安装：

In [ ]:
%pip install -v -r requirements/apex.txt

## CD

定位到工作目录，根据具体情况决定哦，不一定是下面的命令

In [1]:
%cd ..

/home/Public/Megatron-LM


## 下载 Checkpoints

文件比较大，根据实际情况选择下载，**不要重复下载**

In [2]:
import os

S3_BUCKET = 'huamei'
CKPTS_DIR = 'checkpoints/345m-hmwebmix-bpe-v2'

S3_CKPTS_DIR = 's3://' + os.path.join(S3_BUCKET, CKPTS_DIR)

In [3]:
%%time

# 复制 latest_checkpointed_iteration.txt
!aws s3 cp \
    {S3_CKPTS_DIR} \
    {CKPTS_DIR} \
    --recursive \
    --exclude "*" \
    --include "latest_checkpointed_iteration.txt"

# 下载后读取最新的 checkpoint iter 名称
iter_step = open(f'{CKPTS_DIR}/latest_checkpointed_iteration.txt').read().strip()
ckpt_dir = f'iter_{iter_step}'

print(f'checkpoint {ckpt_dir}')

s3_ckpt_dir = os.path.join(S3_CKPTS_DIR, ckpt_dir)
local_ckpt_dir = os.path.join(CKPTS_DIR, ckpt_dir)

print(f'sync: {s3_ckpt_dir} -> {local_ckpt_dir}')
    
# 同步最新的 Checkpiont
!aws s3 sync \
    s3://huamei/hmgpt2-checkpoints/345m-hmwebmix-bpe-v2/iter_0230000 \
    ./checkpoints/345m-hmwebmix-bpe-v2/iter_0230000


checkpoint iter_230000
sync: s3://huamei/checkpoints/345m-hmwebmix-bpe-v2/iter_230000 -> checkpoints/345m-hmwebmix-bpe-v2/iter_230000
CPU times: user 27.4 ms, sys: 21.1 ms, total: 48.5 ms
Wall time: 2.26 s


## Environment Variables

- 用哪个/些 GPU?

In [ ]:
%env CUDA_VISIBLE_DEVICES 0

## Importings

In [4]:
import copy
import csv
import json
import math
import os
import random
import sys
import time
from contextlib import closing
from itertools import chain, compress
from functools import partial
from multiprocessing import Pool
from types import SimpleNamespace

import numpy as np
import torch
import torch.nn.functional as F
from tqdm.auto import tqdm, trange

import mpu
from data_utils.tokenization import SentencePieceTokenizer, make_tokenizer
from pretrain_gpt2 import get_masks_and_position_ids
from predict_gpt2 import initialize_distributed, prepare_tokenizer, set_random_seed, setup_model, get_token_stream

## Args

In [5]:
args = SimpleNamespace(
    # Model arguments
    num_layers=24,
    hidden_size=1024,
    num_attention_heads=16,
    max_position_embeddings=1024,
    vocab_size=None,
    make_vocab_size_divisible_by=128,
    attention_dropout=0.1,
    hidden_dropout=0.1,
    # Train/valid/test data arguments.
    seq_length=1024,
    model_parallel_size=1,
    tokenizer_model_type='bert-large-uncased',
    tokenizer_type='GPT2BPETokenizer_CN',
    tokenizer_path="./data/spm/gpt2_huamei_corpus_bpe_32k_v2.model",
    cache_dir=None,
    # Training arguments.
    load='./checkpoints/345m-hmwebmix-bpe-v2/',
    seed=1234,
    checkpoint_activations=None,
    checkpoint_num_layers=1,
    finetune=None,
    no_load_optim=None,
    no_load_rng=None,
    resume_dataloader=None,
    fp16=True,
    hysteresis=2,
    loss_scale=None,
    loss_scale_window=1000,
    min_scale=1,
    distributed_backend='nccl',
    DDP_impl='local',
    local_rank=None,
    reset_position_ids=None,
    reset_attention_mask=None,
    eod_mask_loss=None, 
    # Text generate arguments.
    recompute=None,
    greedy=False,
    top_p=0.0,
    top_k=0,
    temperature=1.0,
    out_seq_length=128,
)

In [6]:
args.cuda = torch.cuda.is_available()
args.rank = int(os.getenv('RANK', '0'))
args.world_size = int(os.getenv("WORLD_SIZE", '1'))

if os.getenv('OMPI_COMM_WORLD_LOCAL_RANK'):
    # We are using (OpenMPI) mpirun for launching distributed data parallel processes
    local_rank = int(os.getenv('OMPI_COMM_WORLD_LOCAL_RANK'))
    local_size = int(os.getenv('OMPI_COMM_WORLD_LOCAL_SIZE'))

    # Possibly running with Slurm
    num_nodes = int(os.getenv('SLURM_JOB_NUM_NODES', '1'))
    nodeid = int(os.getenv('SLURM_NODEID', '0'))

    args.local_rank = local_rank
    args.rank = nodeid*local_size + local_rank
    args.world_size = num_nodes*local_size

args.model_parallel_size = min(args.model_parallel_size, args.world_size)
if args.rank == 0:
    print('using world size: {} and model-parallel size: {} '.format(
        args.world_size, args.model_parallel_size))

args.dynamic_loss_scale = False
if args.loss_scale is None:
    args.dynamic_loss_scale = True
    if args.rank == 0:
        print(' > using dynamic loss scaling')

# The args fp32_* or fp16_* meant to be active when the
# args fp16 is set. So the default behavior should all
# be false.
if not args.fp16:
    args.fp32_embedding = False
    args.fp32_tokentypes = False
    args.fp32_layernorm = False


using world size: 1 and model-parallel size: 1 
 > using dynamic loss scaling


## Init

### 初始化函数/全局变量

In [7]:
tokenizer = None
model = None

def initialize():
    global model, tokenizer

    # Disable CuDNN.
    torch.backends.cudnn.enabled = False

    # Pytorch distributed.
    initialize_distributed(args)

    # Random seeds for reproducability.
    set_random_seed(args.seed)

    # get the tokenizer
    tokenizer = prepare_tokenizer(args)

    # Model, optimizer, and learning rate.
    model = setup_model(args)

    args.device = torch.cuda.current_device()

    # setting default batch size to 1
    args.batch_size = 1

    assert mpu.get_model_parallel_rank() == 0

### 主进程初始化

In [8]:
%%time

initialize()

> initializing model parallel with size 1
> initializing model parallel cuda seeds on global rank 0, model parallel rank 0, and data parallel rank 0 with model parallel seed: 3952 and data parallel seed: 1234
prepare tokenizer done
building GPT2 model ...
 > number of parameters on model parallel rank 0: 336128000
global rank 0 is loading checkpoint ./checkpoints/345m-hmwebmix-bpe-v2/iter_0230000/mp_rank_00/model_optim_rng.pt
  successfully loaded ./checkpoints/345m-hmwebmix-bpe-v2/iter_0230000/mp_rank_00/model_optim_rng.pt
CPU times: user 11.8 s, sys: 3.58 s, total: 15.4 s
Wall time: 15.4 s


## Inference functions

In [9]:
def infer_tokens_generative(context_tokens, model, tokenizer):
    context_length = len(context_tokens)
    token_stream = get_token_stream(model, [context_tokens], tokenizer, args)   
    for i, (output_tokens, _) in enumerate(token_stream):
        if context_length + i >= args.seq_length:
            break
        ids = output_tokens.cpu().numpy().tolist()[0]
        yield ids[-1]


def infer_text_generative(contex_text, model, tokenizer):
    contex_text = contex_text.strip()
    context_tokens = tokenizer.EncodeAsIds(contex_text).tokenization
    context_length = len(context_tokens)

    token_stream = get_token_stream(model, [context_tokens], tokenizer, args)
    
    for i, (output_tokens, _) in enumerate(token_stream):
        if context_length + i >= args.seq_length:
            break
        ids = output_tokens.cpu().numpy().tolist()[0]
        s = tokenizer.DecodeIds([ids[-1]])
        yield s


## 预测试试看

In [13]:
contex_text = '我喜欢'

print(contex_text)
print('==========')
print()

N = 3
for i in range(N):
    print(f'({1+i})/{N}\t', end='')
    for s in infer_text_generative(contex_text.strip(), model, tokenizer):
        print(s, end='')
    print()
    print()

我喜欢

(1)/3	刘涛,不喜欢她和高圆圆。为什么我喜欢高圆圆离开后,那个高圆圆会这么空洞。其实不是某个神奇而迷人的女性,因为每个人心中都有一个高圆圆,如果愿意,是自己对她的刻意追求以及自卑。高圆圆夺得金鹰女神之后,真实的她是多么好奇怪。先来做些科普吧。高圆圆最出名的笑容是看着别人的眼睛露出温柔,似乎这个世上还没有一个人比她更温柔了,仿佛的眼睛里只剩下柔情蜜意。这是美人如瓶,骨如锉的特点。重点来了,你以为的

(2)/3	这台曾经大闹画拍卖的相机,首先是因为它保养我父亲13年的时间。用3年的时间拿到一块当时全国产相机里最高的水准:3500像素F1.7广角、3500像素长焦以及100万像素4K清胶片。与其他入门级相机相比,F1.7的画质新奇,而且更加配备许多功能强大的人像模式,OCAP机型的STMMN动画可以连续撕下电池底座,让画面更加美轮美奂。首先,单击相机菜单的Joy-Conn

(3)/3	这样的漫漫长夜,但我还是忍不住第二场剧目的巡演邀请他们,让我走出深渊,去彼此看一个“有故事”(对于女主来说)的夜色,所以我的生意就越来越多。后台一直在说女孩子长大不是很漂亮,但常常在粉丝面前得到一句抱歉,如果是一个男孩子,他会说这句话,让我此刻内心平静。跑圈,上验光,做手术,甚至拍走路的戏要求也变多了,对剧知道的越来越多,对角色也越来越熟悉这个时候的《太子妃升职记》找到我,找到我想做演员,也很满意。但是之前



## 对一组心理/情绪相关文字进行续写

In [14]:
input_texts = [
    '宝贝“啃指甲”是为什么呢？宝贝是缺铁、缺锌了吗？宝贝是肚子里有虫吗？这指甲里好多细菌宝宝都吃下去怎么是好？怎么能让宝宝不“啃指甲”呢？家长应该怎么应对呢？',
    '几年前，英国就有专家向国会递交报告，建议政府制定法律禁止家长让年龄低于3岁的儿童看电视。奇幻小说《查理和巧克力工厂》有句台词：“千万、千万、千万别让孩子，靠近你的电视，最好是别购买、安装，这最最愚蠢的东西”。',
    '婚姻是一座炼狱，因为在这场婚姻里夹杂着各种人际关系，经过这些人际关系，人才能慢慢心智成熟起来。而能够把这些人际关系处理好的，往往都是生活中的高手。那么夫妻双方该如何经营婚姻呢？这里就提供一些方法供参考。',
]

In [16]:
# args.recompute=True
# args.top_p=0.0
# args.top_k=0
# args.temperature=1.0

n_gen = 2

for txt in input_texts:
    context_tokens = tokenizer.EncodeAsIds(txt).tokenization
    print(txt)
    print()
    for i in range(n_gen):
#         args.temperature=random.gauss(0.95, 0.05)
        print(f'{i}: ', end='')
        s_pred = ''
        for s in infer_text_generative(txt, model, tokenizer):
            print(s, end='')
        print(os.linesep)
    print()
    print('=' * 100)
    print()
    

宝贝“啃指甲”是为什么呢？宝贝是缺铁、缺锌了吗？宝贝是肚子里有虫吗？这指甲里好多细菌宝宝都吃下去怎么是好？怎么能让宝宝不“啃指甲”呢？家长应该怎么应对呢？

0: 是教孩子养成好习惯还是强迫孩子多吃点零食?通常,初二生长的宝宝,特爱心高胆大性情急躁,因此,家长最好就不要强迫孩子啃指甲。可物理干燥除甲坏习惯也或可引起阴痒影响食欲,增加肠胃负担,应该在散完水分后即用富含碘的食物涂擦,有些孩子用手抚摸指甲,往往会抓破手指皮肤,而产生细菌感染。家长还应教给孩子注意卫生,如保持皮肤干燥。如果孩子的指甲很扁平,建议到医院检查一下

1: 下面,咱们就一起来看一看吧,有关于宝宝胃肠道人体内是什么细菌的。一、维生素类2岁之前婴幼儿消化道的标本物质构成与成人相同,基本不全面,而且还容易摄入脂肪。同时,大多数微生物都渗透到五官之中,极容易与粘膜结合侵入胃肠道内,引起肠刺激反应缺铁,影响脂肪代谢的主因,大多由缺铁而引起。辅食中的铁元素缺乏,影响脑脊液的合成,容易引起中耳炎等传染病。包囊内有凝集素,



几年前，英国就有专家向国会递交报告，建议政府制定法律禁止家长让年龄低于3岁的儿童看电视。奇幻小说《查理和巧克力工厂》有句台词：“千万、千万、千万别让孩子，靠近你的电视，最好是别购买、安装，这最最愚蠢的东西”。

0: 有点教育专家认为婴幼儿对应的是摇篮一样,可以说是一个“摇篮大师”,因为婴儿时期的教育是优于成人的。因为婴儿时期的幼儿可以说都是模仿成人的,当儿童逐渐长大,或者婴儿从两个方面,尤其是从身体状况、上幼儿园这些方面可能远不如成人此前,「失去对世界的认识、由此而产生的对外界新的经验以及和成人的交往和交流的能力」,而只有通过父母对婴儿的早期教育,并走向社会而积累下一定的社会资源时,才能有效的培养儿童的这一特点。这个国家人口基数并不大,孩子逗小孩子玩的时间应该

1: 那能不能让孩子远离低龄电视?小编死磕专家,凭精神数了数,孩子出自哪里就自行走脑。首先跟大家聊聊在成人眼中的电视这份礼仪。英国的理由比较灵活,有些是出于礼貌仪态,比如电视机的功率要低于路人的照明;有些是别绊着“孩子的眼”,比如上面的问题,如果孩子正好在看电视,请他到旁边站一会儿,一个多小时后再拿回来,流氓关系一下子淡了。看了英国人写进法律的拔高法律要求且符合道德规范的六类规定后,你又多了



婚姻是一座炼狱，因为在这场婚

## Test

使用 test 语料，从中随机打断，并预测下文，比较原文与预测结果！

随机选 N 个

In [12]:
N = 100

input_file = './data/hmwebmix/hmwebmix.test.json'
output_file = f'./data/hmwebmix/hmwebmix.test.{iter_step}.{args.out_seq_length}.{N}.tsv'

print(f'output_file={output_file}')

total = sum(1 for _ in tqdm(open(input_file)))
print(f'Test 数据总数: {total:,d}')

assert total >= N

print(f'Test 采样数: {N:,d}')

mask = np.zeros(total, dtype=int)
mask[:N] = 1
np.random.shuffle(mask)

samples = []
with open(input_file) as fp:
    reader = compress(fp, mask)
    for line in tqdm(reader, 'sample', total=N):
        line = line.strip()
        if not line:
            continue
        text = json.loads(line)['text']
        samples.append(text)
random.shuffle(samples)

with open(output_file, 'w') as fp:
    writer = csv.writer(fp, delimiter='\t')
    for context_txt in tqdm(samples, 'infer'):
        context_ids = tokenizer.EncodeAsIds(context_txt).tokenization
        context_len = len(context_ids)
        idx = round(random.gauss(context_len*0.5, context_len*0.1))
        input_ids = context_ids[:idx]
        label_ids = context_ids[idx:]
        infer_ids = [id_ for id_ in infer_tokens_generative(input_ids, model, tokenizer)]
        row = [
            tokenizer.DecodeIds(ids)
            for ids in (input_ids, label_ids, infer_ids)
        ]
        writer.writerow(row)


10000it [00:00, 108124.58it/s]
infer:   0%|          | 0/100 [00:00<?, ?it/s]

output_file=./data/hmwebmix/hmwebmix.test.230000.128.100.tsv
Test 数据总数: 10,000
Test 采样数: 100


infer: 100%|██████████| 100/100 [06:20<00:00,  3.80s/it]


## 预测一组 QA

In [ ]:
input_texts = [
    (
        '最近晚上睡不着觉，但很困，总觉得不会睡觉？',
        '最近一段时间，总是晚上睡不着觉，但是却很困，没次睡觉心理都在不自觉的想着该如何去睡觉，每次想着想着就完全清醒了，失眠特别严重，感觉身体快要透支了，很无助，还连累家人一起跟着担心'
    ),
    (
        '大一，谈了将近一年的女友突然提出分手我该怎么办？',
        ' 某一天晚上她突然给我打电话说我不是她喜欢的那种类型，在电话中她多次提到自己的性格不好，说“我所看到的都是加了“滤镜”的效果，其实她并没有我想得那么好，今后不准备结婚”。（她以前谈过一次恋爱，对于那场恋爱她一直都铭记在心）接完电话我很迷茫和困惑，因为之前她愿意跟我分享她的生活中的不如意和困难，突然的转变让我措手不及。同时我也很自责，没有准确的表达出自己的意思和情感，在与她的对话中我常常感到无力和无助。我到底该怎么办？我真的很爱她，想和她度过一生。'
    ),
    (
        '男人出轨',
        '我的意思是，出轨者会问自己，我为什么要这么做，被背叛的伴侣会问，你为什么要这样对我?'
    ),
    (
        '我的奇葩婚姻，老公说为了孩子好就是不离婚？',
        '我的婚姻最近好像一塌糊涂，可是老公不离婚，我甚至有想过死？我是不是得了抑郁症？和老公从2017年开始就吵吵闹闹，一吵架就分居，分居一两个月老公可以对我和孩子不问不闻，到时间了他就自己又搬回来，但是回来也不和我沟通，就算是上次是他动手，他也不道歉，他特别喜欢冷战。现在和老公又发展到了无性，而且双方家庭也合不来，我和他家人不敢来往，他家人对我要求高，希望天天在家做饭带娃伺候老公，他现在和我父母也不来往，从来不回家吃饭，要等我父母走了他才回来，每天很晚回家，去哪从来不告诉我，我知道他有婚外性，但他從來不承認，还说我是神经病，我還被他傳播上了高危型的HPV，以后是死是活都不确定，我特别想要离婚，我感觉和他生活快要窒息了，他整天拉着个脸，要么对我不闻不问，要么一开口说话就是责怪的语气，但他就是不离婚，说是为了孩子好，还让我不要发神经，我真的不知道该怎么和他相处了。他性格很古怪'
    ),
    (
        '妻子在钱的问题上不坦诚，如何处理好家庭经济问题？',
        '和老婆因为意外怀孕结婚，两家谈婚论嫁时闹得不愉快，我家出了一套大户型的房子，一台豪华车（当然这些都是婚前财产），然后给了40万装修婚房，同时给女方10万彩礼。而女方一开始的态度是一分钱没有，后来迫于压力给了50万现金，我老婆名下既没有房也没有车。婚后的钱一直都是给我老婆管理，包括我的工资奖金还有结婚的份子钱等等。我老婆并没有把这些钱看做是家庭财产，就算其中已经掺入了很多我的钱，还是认为全部都是她自己的钱，我只是知道她的卡密码而已，但是卡在哪里，她花了多少钱，花在哪里了，有没有借钱给别人，我通通不知道，这让我很不安。况且她没有理财观念，我曾经三番五次跟她说了理财的重要性，她就只是沉默不语，也不知道是听不懂，还是装傻实际上自己去做了理财但是没把收入情况告诉我，我们多次吵架都是为了钱，我觉得夫妻两个在这个问题上不能坦诚是个大问题，我现在很想经济分开，但是不知道怎么做才比较妥当'
    ),
    (
        '30岁男生，相亲屡次不顺，我该怎么办呢？',
        '30岁男生还单身，相亲对象不少，可是次次都以失败收场，自己长相还行，就是不太会说话，和女生聊天刚开始聊的挺不错，没聊几天女的就不回复了，爱答不理，心里郁闷的晚上都想哭，我该怎么办'
    ),
    (
        '30岁女生，离婚念头挥之不去，他为何计划婚内出轨？',
        '此刻有点儿失眠心情烦躁，我想了很久没有想明白，为什么他会计划着婚内出轨，……心里，我基本上认定原因应该是我身材不怎么好，胸部不够吸引人，因为婚后发现了在婚前，他约P了一个又肥又长相不佳的女人。我以为就是因为对方胸部傲人。在壹心理一些文章的引导下，我突然明白，源头是性生活不和谐。婚前同居一年多，到婚后2年，性生活基本就是，直奔主题，他一手拿着手机欣赏着他的成人影片一边爱爱，很少会主动关心我的感受，也很少理会我让他收起手机的要求。而且另一方面，他基本每晚都会要求我用嘴亲亲，他自己则拿着手机看片或者无关紧要的东西，让我经常觉得自己就是一个娃娃一个工具。以前我主动跟他说起过几次，然而也只是不了了之。后来我基本对性生活没有兴趣，也很反感为他口J。而且，日常生活变得频繁小吵冷战，很少有之前无话不谈的亲密，我不知道我们还有没有重归于好的机会……'
    ),
    (
        '29岁哺乳期，和公婆因带孩子问题每天崩溃，怎么办？',
        '怀孕她就没怎么管，备产各种东西都是自己准备。产后直接是妈妈照顾的。产后半年回来要上班，我爸妈又没退休，无奈下还是他们来照顾。他们方法不对，连个衣服都洗不干净，更不用提孩子的饭了。每天我都自己趁下班时间给孩子准备饭和各种东西。最怕的是习惯，比如她喜欢喂孩子各种炒菜，我说好几次一岁之内吃盐不好就是不听，还故意喂。或者孩子的奶瓶，经常不盖盖子让他把玩，说不卫生，也不搭理。孩子吃饭的碗和勺子永远是扔在各种桌子椅子角落，用的时候直接拿起来用也不洗，洗脸洗脚水每次告诉温度差不多就好了，就是不听，偏要一会冷一会烫。来我家被子也不叠，锅永远是不会洗的，下一顿加水加米接着煮。我老公又每天忙的不在家，我已经要崩溃了'
    ),
    (
        '婚后分居三四年，有过家庭矛盾，我的婚姻是否该继续？',
        '婚后分居三四年，和老公之间发生过一些家庭矛盾，可以看我另外一篇疑问。我们上半年闹离婚，后来他又主动和好，但是我们之前除了新婚，几乎很少有夫妻生活，他说一靠近我就觉得心里像一堆蚂蚁抓挠，特别难受。他说这是他得心理问题，小时候受到唾骂和凌辱，造成了心理阴影。我们俩都觉得生活的很痛苦，但为了孩子没有离婚。我跟他在一起就觉得拘束，不自在。向往的温馨家庭生活也得不到，我很累，我应该坚持离婚吗？他这种心理疾病是真的还是借口呢？'
    ),
#     (
#         '',
#         '新 闻 大S自嘲上辈子应是儿子的“杀父仇人”，笑说：“不然我怎么一生他就快死了，他是来报仇的吧，而且我一抱他就哭，到三个月后才好一点。”她生下一双儿女集满“好”字，感到非常满足，是否再拼第三胎？大S笑说：“不可能！我现在尽量和老公分房睡。” 评析起初，大家并不看好大S与汪小菲这段婚姻。姐弟恋是一回事儿，更多还是觉得有些不够门当户对。如今，女儿与儿子陆续出生。凑足“好”字的两人，幸福得就像花儿开放。俏江南的变化，对大S与汪小菲毫无影响。男的负责赚钱养家，女的负责照顾孩子开心美满。'
#     )
]


In [ ]:
# args.recompute=True
# args.top_p=0.0
# args.top_k=0
# args.temperature=1.0

n_gen = 2

for title, text in input_texts:
    txt = f'{title}<sep>{text}<|endoftext|>'
    context_tokens = tokenizer.EncodeAsIds(txt).tokenization
    print(title)
    print(text)
    print()
    for i in range(n_gen):
#         args.temperature=random.gauss(0.95, 0.05)
        print(f'{i}: ', end='')
        s_pred = ''
        for s in infer_text_generative(txt, model, tokenizer):
            print(s, end='')
        print(os.linesep)
    print()
    print('=' * 100)
    print()
    

## 手动续写测试

定义问题

In [ ]:
question = (
'''
婚后分居三四年，有过家庭矛盾，我的婚姻是否该继续？
''',
'''
婚后分居三四年，和老公之间发生过一些家庭矛盾，可以看我另外一篇疑问。
我们上半年闹离婚，后来他又主动和好，但是我们之前除了新婚，几乎很少有夫妻生活，他说一靠近我就觉得心里像一堆蚂蚁抓挠，特别难受。他说这是他得心理问题，小时候受到唾骂和凌辱，造成了心理阴影。
我们俩都觉得生活的很痛苦，但为了孩子没有离婚。我跟他在一起就觉得拘束，不自在。
向往的温馨家庭生活也得不到，我很累，我应该坚持离婚吗？
他这种心理疾病是真的还是借口呢？
'''
)

**这在里写我的回答**

In [ ]:
# 在这里写一部分
answer = '''
我觉得这是你
'''


**执行下面的 cell ，帮我写把上面这句写完，或者写下一句**

In [ ]:
n_pred = 3

for i in range(n_pred):
    
    txt = '<sep>'.join(s.strip() for s in question) + '<|endoftext|>' + '<bos>' + answer.strip()
    context_tokens = tokenizer.EncodeAsIds(txt).tokenization
    
    print(f'{i+1}: ', end='')
    s_pred = ''
    with closing(
        infer_text_generative(txt, model, tokenizer)
    ) as generator:
        for s in generator:
            if s:
                print(s, end='')
                if s[-1] in ('.。!！?？'):
                    break
    print()